# Titel
### Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import permutations
from collections import deque
from typing import Union

### Functions

In [2]:
######## Conversions ###################################

def toStr(vec) -> str:
    return ''.join(map(str, vec))

def toInt(vec) -> int:
    return int("".join(str(i) for i in vec), 2)

def toBinVec(x, l=0) -> list:                # l=0 -> Dynamische Länge, l=10 -> Fixe Länge
    result = [int(i) for i in bin(x)[2:]]
    missing_zeros = l-len(result)
    if (missing_zeros > 0):
        for i in range(missing_zeros):
            result.insert(0, 0)
    return result

######### Opertations #####################################

def hammingweight(vec) -> int:
    return np.count_nonzero(vec == 1)

def combinations(iterable, r):
    pool = tuple(iterable)
    n = len(pool)
    for indices in permutations(range(n), r):
        if sorted(indices) == list(indices):
            yield tuple(pool[i] for i in indices)
            
def binary_permutations(lst: []) -> [[]]:
    result = []
    for comb in combinations(range(len(lst)), lst.count(1)):
        permutation = [0] * len(lst)
        for i in comb:
            permutation[i] = 1
        result.append(permutation)
    return result

def rotate_left_vec(vec) -> list:
    shifted = vec.copy()
    temp = deque(shifted)
    temp.rotate(-1)
    return list(temp)

########## Search Functions ################################
def find_extrema(vec) -> Union[int,int,int,list,list]:
    L = len(vec)
    vecToInt = toInt(vec)
    min = vecToInt
    max = 0
    odd_max = 0           ####????
    vec_min = vec
    vec_max = vec
    shifted = vec.copy()
    for i in range(L):
        if vecToInt < min:
            min = vecToInt
            vec_min = shifted
        if vecToInt > max:
            max = vecToInt
            vec_max = shifted
        temp = deque(shifted)
        temp.rotate(-1)
        shifted = list(temp)
        vecToInt = toInt(shifted)
    return min, max, odd_max, vec_min, vec_max

def find_rotational_distance(vec1, vec2) -> Union[int,int]:
    L = len(vec1)
    vec1ToInt = toInt(vec1)
    vec2ToInt = toInt(vec2)
    left_shifts = 0
    shifted = vec1.copy()
    for i in range(L):
        if toInt(shifted) == vec2ToInt:
            left_shifts = i
            break
        temp = deque(shifted)
        temp.rotate(-1)
        shifted = list(temp)
    return left_shifts, L-left_shifts   #### Links ; Länge-Links = Rechts

########## Calculations ######################################
def calculate_c(n: int, U: int) -> int:
    return 2**n-3**U

def calculate_z(parity_vector: []) -> int:
    indices = []
    for i, entry in enumerate(parity_vector, start=0):
        if entry == 1:
            indices.append(i)
    result = 0
    U = len(indices)
    for i, entry in enumerate(indices, start=0):
        result = result + 3**(U-(i+1)) * 2**(entry)
    return result

def z_inverse(z, l) -> int:
    i = 1
    while True:
        vec = toBinVec(i, l)
        if calculate_z(vec) == z and len(vec) == l:
            return i
        i += 1
        
def getCycle(start):
    cycle = np.array(calculate_z(start))
    pos = start
    for i in range(8):
        pos = rotate_left_vec(pos)
        cycle = np.hstack([cycle, calculate_z(pos)])
    return cycle

### Prepare Data

In [3]:
def setInput(l) -> list:                
    delay = np.zeros((l,), dtype=int)
    for i in range(2**l-1):
        delay = np.vstack([delay, toBinVec(i+1, l)])
    INPUT = delay
    return INPUT

def setData(B_vec) -> list:
    EXP = find_extrema(B_vec)                     # Extrema
    l = len(B_vec)                                # Länge
    q = find_rotational_distance(EXP[3], EXP[4])  # Rotating distance - Both ?
    r = q[0]                                      # Rotating distance - Leftrotation ?
    U = hammingweight(B_vec)                      # Hamming weight
#     if U == 0:
#         DarrellCox = 0
#     else:
#         DarrellCox = (l*r)/U + 1                      # Darrell Cox        
    div_modular = (U*r+1)/l                       # Division   ?
    div_integer = EXP[1] - EXP[0]                 # Differenz Maxima
    cycle = getCycle(B_vec)                       # z(B) Cycle
    c = calculate_c(l, U)                         # "c"
    
    ####### min ### max ### B_min # B_max # l  r  U # (U*r+1)/l # B_max-B_min # cycle # c
    DATA = [[EXP[0], EXP[1], EXP[3], EXP[4], l, r, U, div_modular, div_integer, cycle, c]]
    return DATA

def getData(l) -> list:
    B = setInput(l)
    INPUT = B[0]      ## Zeilen auslesen
    DATA = setData(INPUT)
    for i in range(2**l-1):
        remain = B[i+1]      ## Zeilen auslesen
        DATA = np.vstack([DATA, setData(remain)])
        INPUT = np.vstack([INPUT, remain])
    return DATA, INPUT

### MAIN

In [4]:
l = 8    ### !16 bit -> 10[min] Runtime
[DATA, INPUT] = getData(l)
print(INPUT[0:10])
print('...')

[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 1 1]
 [0 0 0 0 0 1 0 0]
 [0 0 0 0 0 1 0 1]
 [0 0 0 0 0 1 1 0]
 [0 0 0 0 0 1 1 1]
 [0 0 0 0 1 0 0 0]
 [0 0 0 0 1 0 0 1]]
...


C:\Users\adrian.AUENLAND\Anaconda3\lib\site-packages\numpy\core\_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


### Summary

In [17]:
df1 = pd.DataFrame(DATA, columns = ['min', 'max', 'B_min', 'B_max', 'l', 'r', 'U', 'Diff', 'B_max-B_min', 'cycle', 'c'])
pd.set_option('display.expand_frame_repr', True)

df1_filterd = df1[(df1.Diff == 1) | (df1.Diff == 2) | (df1.Diff == 3) | (df1.Diff == 4)]
df1_sorted = df1_filterd.sort_values(by=['Diff', 'r','B_max-B_min'])

# df1[0:65536]
df1_sorted

,min,max,B_min,B_max,l,r,U,Diff,B_max-B_min,cycle,c
127,127,254,"[0, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 0]",8,1,7,1,127,"[4118, 2059, 2123, 2219, 2363, 2579, 2903, 338...",-1931
191,127,254,"[0, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 0]",8,1,7,1,127,"[3389, 4118, 2059, 2123, 2219, 2363, 2579, 290...",-1931
223,127,254,"[0, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 0]",8,1,7,1,127,"[2903, 3389, 4118, 2059, 2123, 2219, 2363, 257...",-1931
239,127,254,"[0, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 0]",8,1,7,1,127,"[2579, 2903, 3389, 4118, 2059, 2123, 2219, 236...",-1931
247,127,254,"[0, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 0]",8,1,7,1,127,"[2363, 2579, 2903, 3389, 4118, 2059, 2123, 221...",-1931
251,127,254,"[0, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 0]",8,1,7,1,127,"[2219, 2363, 2579, 2903, 3389, 4118, 2059, 212...",-1931
253,127,254,"[0, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 0]",8,1,7,1,127,"[2123, 2219, 2363, 2579, 2903, 3389, 4118, 205...",-1931
254,127,254,"[0, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 0]",8,1,7,1,127,"[2059, 2123, 2219, 2363, 2579, 2903, 3389, 411...",-1931
1,1,128,"[0, 0, 0, 0, 0, 0, 0, 1]","[1, 0, 0, 0, 0, 0, 0, 0]",8,7,1,1,127,"[128, 64, 32, 16, 8, 4, 2, 1, 128]",253
2,1,128,"[0, 0, 0, 0, 0, 0, 0, 1]","[1, 0, 0, 0, 0, 0, 0, 0]",8,7,1,1,127,"[64, 32, 16, 8, 4, 2, 1, 128, 64]",253


In [16]:
df2 = df1.copy()
df2_drop = df2.drop(columns=['B_min', 'B_max', 'l', 'r', 'U', 'Diff', 'B_max-B_min', 'cycle', 'c'])
df2_diff = df2_drop.diff()
df2_renamed = df2_diff.rename(columns={'min': 'Delta_min', 'max': 'Delta_max'})




# df2_diffquot = df2_renamed['Differentquotient'] = df2_renamed['Delta_min'] / df2_renamed['Delta_max']
# df2_diff['e'] = pd.Series(4, index=df1.index)
# df2_diff[0:20]

result = pd.concat([df1, df2_renamed], axis=1)
result[0:25]

,min,max,B_min,B_max,l,r,U,Diff,B_max-B_min,cycle,c,Delta_min,Delta_max
0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]",8,0,0,0.125,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0]",255,NaN,NaN
1,1,128,"[0, 0, 0, 0, 0, 0, 0, 1]","[1, 0, 0, 0, 0, 0, 0, 0]",8,7,1,1,127,"[128, 64, 32, 16, 8, 4, 2, 1, 128]",253,1,128
2,1,128,"[0, 0, 0, 0, 0, 0, 0, 1]","[1, 0, 0, 0, 0, 0, 0, 0]",8,7,1,1,127,"[64, 32, 16, 8, 4, 2, 1, 128, 64]",253,0,0
3,3,192,"[0, 0, 0, 0, 0, 0, 1, 1]","[1, 1, 0, 0, 0, 0, 0, 0]",8,6,2,1.625,189,"[320, 160, 80, 40, 20, 10, 5, 131, 320]",247,2,64
4,1,128,"[0, 0, 0, 0, 0, 0, 0, 1]","[1, 0, 0, 0, 0, 0, 0, 0]",8,7,1,1,127,"[32, 16, 8, 4, 2, 1, 128, 64, 32]",253,-2,-64
5,5,160,"[0, 0, 0, 0, 0, 1, 0, 1]","[1, 0, 1, 0, 0, 0, 0, 0]",8,5,2,1.375,155,"[224, 112, 56, 28, 14, 7, 134, 67, 224]",247,4,32
6,3,192,"[0, 0, 0, 0, 0, 0, 1, 1]","[1, 1, 0, 0, 0, 0, 0, 0]",8,6,2,1.625,189,"[160, 80, 40, 20, 10, 5, 131, 320, 160]",247,-2,32
7,7,224,"[0, 0, 0, 0, 0, 1, 1, 1]","[1, 1, 1, 0, 0, 0, 0, 0]",8,5,3,2,217,"[608, 304, 152, 76, 38, 19, 143, 329, 608]",229,4,32
8,1,128,"[0, 0, 0, 0, 0, 0, 0, 1]","[1, 0, 0, 0, 0, 0, 0, 0]",8,7,1,1,127,"[16, 8, 4, 2, 1, 128, 64, 32, 16]",253,-6,-96
9,9,144,"[0, 0, 0, 0, 1, 0, 0, 1]","[1, 0, 0, 1, 0, 0, 0, 0]",8,4,2,1.125,135,"[176, 88, 44, 22, 11, 140, 70, 35, 176]",247,8,16


#### Export Data

In [36]:
# df1_sorted.to_excel("Output_Data_8_bit.xlsx",
#              sheet_name='Filterd_Diff_and_Sorted') 
    
with pd.ExcelWriter("Output_Data_8_bit.xlsx", engine="openpyxl", mode="a") as writer:
    df1.to_excel(writer, sheet_name="Sheet_name_2", startrow=1, startcol=1)

### Plotting -> Tableu

In [26]:
x = []
y = []
y_offset = []
#color_scheme=[]                     
colors = []
i = 0
for index, row in df_plot.iterrows():  ## df_plot -> Pandas
    x.append(i)
    yo = row['min']
    y_offset.append(yo)
    y.append(row['max']-yo)
    colors.append(color_scheme[row['r']-1])
    i += 1
    
plt.bar(x, y, bottom=y_offset, color=colors)
plt.show()

plt.scatter(df_plot["diff_min"].tolist(), df_plot["diff_max"].tolist(), s=1)
plt.show()